In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [14]:
DATA = []



for page in range(1, 8): #границы отрезка зависят от кол-ва страниц результатов 
    url0=f"https://grls.rosminzdrav.ru/GRLS.aspx?RegNumber=&MnnR=&lf=&TradeNmR=&OwnerName=эллара&MnfOrg=&MnfOrgCountry=&isfs=0&regtype=1%2c6&pageSize=10&order=Registered&orderType=desc&pageNum={page}"
    
    l = requests.get(url0)
    soup0 = BeautifulSoup(l.text, 'lxml')
    
    for i in soup0.findAll('tr', class_='hi_sys'):
        url = "https://grls.rosminzdrav.ru/Grls_View_v2.aspx?routingGuid="+i.get('onclick').split("'")[1] #с каждой строки идет переход на отдельную страницу
        r = requests.get(url)
        sleep(3) #чтобы не перегружать и не нарваться на блокировку
        soup = BeautifulSoup(r.text, 'lxml')
            
        lines = soup.findAll('tr', class_ = 'ter1')
        RegNr = lines[0].find(id='ctl00_plate_RegNr').get('value')
        RegDate = lines[0].find(id='ctl00_plate_RegDate').get('value')
        trade_name = lines[2].find(id='ctl00_plate_TradeNmR').get('value')
        Innr = lines[3].find(id='ctl00_plate_Innr').text
        forma = lines[4].find('tr', class_="hi_sys").find('td').text.strip() 
        codeATX = lines[9].find('tr', class_="hi_sys").find('td').text

        data1 = [RegNr, RegDate, trade_name, Innr, forma, codeATX]
                
        if lines[10].findAll('tr', class_='hi_sys'):
            
            for i in lines[10].findAll('tr', class_='hi_sys'):
        
                substation = i.findAll('td')[0].text
                manufacturer = i.findAll('td')[2].text
                adress = i.findAll('td')[3].text
                ND = i.findAll('td')[6].text
                DATA.append(data1 + [substation, manufacturer, adress, ND])
        else:
            DATA.append(data1 + ['~', '~', '~', '~'])


In [15]:
header = ["number", "date1", "trade_name", "Innr", "forma", "code_ATX", "substation", "manufacturer", "adress", "ND"]
df = pd.DataFrame([DATA][0], columns=header) 


for i in range(len(DATA[1::])):
   
    df.loc[i] = DATA[i]
df.to_excel(r'C:\Users\Катерина\Desktop\ellara.xlsx', encoding='utf-16') 

C:\Users\Катерина\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
